In [1]:
import numpy as np  
import matplotlib.pyplot as plt
import cv2
import os
import numba
from numba import njit
from collections import deque
import imageio
from datetime import datetime

In [2]:
def rotate_points(points, angle_deg):
    angle_rad = np.radians(angle_deg)
    rotation_matrix = np.array([[np.cos(angle_rad), -np.sin(angle_rad)],
                                [np.sin(angle_rad),  np.cos(angle_rad)]])
    rotated_points = np.dot(points, rotation_matrix.T)
    return rotated_points
    
def process_dynamic_actor(actor_data):
    # get the actor x, y, yaw, and velocity
    x,y,_,_,yaw,_ = actor_data[2]
    vx,vy,_ = actor_data[3]
    _,_,_,bb_width,bb_height,_,_ = ego_data[6]
    
    bb_coords = np.array([[-bb_width, -bb_height], [bb_width, -bb_height], [bb_width, bb_height], [-bb_width, bb_height]])    
    rotated_bbox = rotate_points(bb_coords, yaw)
    
    return (x,y), (vx,vy), rotated_bbox
    
def discretize_coordinates(x_coords, y_coords, bins):
    # descretize the shifted rotated bbox
    x_ = np.digitize(x_coords, bins)
    y_ = np.digitize(y_coords, bins)
    
    return np.vstack((x_, y_)).T
    

In [3]:

@numba.jit(nopython=True)
def get_line_pixels(x1, y1, x2, y2):
    """Bresenham's Line Algorithm"""
    dx = x2 - x1
    dy = y2 - y1

    xsign = 1 if dx > 0 else -1
    ysign = 1 if dy > 0 else -1

    dx = abs(dx)
    dy = abs(dy)

    if dx > dy:
        xx, xy, yx, yy = xsign, 0, 0, ysign
    else:
        dx, dy = dy, dx
        xx, xy, yx, yy = 0, ysign, xsign, 0

    D = 2*dy - dx
    y = 0

    line = np.empty((dx + 1, 2), dtype=np.int64)
    for x in range(dx + 1):
        line[x] = [x1 + x*xx + y*yx, y1 + x*xy + y*yy]
        if D >= 0:
            y += 1
            D -= 2*dx
        D += 2*dy
    return line[:, 1], line[:, 0]

@numba.jit(nopython=True)
def get_border_pixels(height, width):
    border_coordinates = np.zeros((2*(height+width-2), 2), dtype=np.int64)
    index = 0
    for j in range(width): # top_border
        border_coordinates[index] = (0, j)
        index += 1
    for j in range(width): # bottom_border
        border_coordinates[index] = (height-1, j)
        index += 1
    for i in range(1, height-1): # left_border
        border_coordinates[index] = (i, 0)
        index += 1
    for i in range(1, height-1): # right_border
        border_coordinates[index] = (i, width-1)
        index += 1
    return border_coordinates

    
@numba.jit(nopython=True)
def ray_trace_fast(mgrid):
    mgrid = 1 - .5*(1 - mgrid)  # make objects black, unknown space gray, and free space white

    # Calculate the center of the grid
    center_y, center_x = mgrid.shape[0] // 2, mgrid.shape[1] // 2

    # Calculate window range as the max of the half of the grid's dimensions
    window_range = max(mgrid.shape[0], mgrid.shape[1]) // 2

    # Calculate border coordinates (this assumes that 'get_border_pixels' takes a width and height as parameters)
    border_coordinates = get_border_pixels(mgrid.shape[1], mgrid.shape[0])

    # Proceed with ray tracing as before
    for i in range(border_coordinates.shape[0]):
        end_y, end_x = border_coordinates[i]
        points_y, points_x = get_line_pixels(center_x, center_y, end_x, end_y)
        for j in range(points_x.shape[0]):
            x, y = points_x[j], points_y[j]
            if mgrid[y, x] != 1:
                mgrid[y, x] = 0
            else:
                break

    return mgrid
    

In [4]:
def create_rgb_image(velocity, vmin=None, vmax=None):
    # Ensure that velocity has the correct shape
    assert len(velocity.shape) == 3 and velocity.shape[2] == 2, \
        "Input array must have shape (height, width, 2)"

    # If vmin or vmax are provided, clip the velocities
    if vmin is not None or vmax is not None:
        velocity = np.clip(velocity, vmin, vmax)

    # Separate the velocity components
    vx, vy = velocity[..., 0], velocity[..., 1]

    # Calculate magnitude and angle from vx and vy
    magnitude = np.sqrt(vx**2 + vy**2)
    angle = (np.arctan2(vy, vx) + np.pi) / (2 * np.pi)  # Normalize to [0, 1]

    # Create an HSV image where:
    # - Hue (H) is the angle (which gives the direction of the vector)
    # - Saturation (S) is always 1 (full color)
    # - Value (V) is the magnitude (which gives the length of the vector)
    hsv_image = np.zeros((*vx.shape, 3))  # 3 for H, S, V
    hsv_image[..., 0] = angle * 180  # OpenCV expects H in range [0, 180]
    hsv_image[..., 1] = 255  # OpenCV expects S in range [0, 255]
    hsv_image[..., 2] = (magnitude / np.max(magnitude)) * 255  # Normalize magnitude and scale to [0, 255]

    # Convert HSV image to RGB
    hsv_image = hsv_image.astype(np.uint8)
    rgb_image = cv2.cvtColor(hsv_image, cv2.COLOR_HSV2BGR)

    return rgb_image
    
    
# Function to create timestamped directories
def create_timestamped_directory(base_dir):
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    timestamped_dir = os.path.join(base_dir, 'dogma_seq_'+timestamp)
    os.makedirs(timestamped_dir, exist_ok=True)
    return timestamped_dir

In [5]:
import os
import numpy as np
import cv2
from collections import deque

folder_path = './tmp'


for item in os.listdir(folder_path):
    
    clip_name = os.path.join(folder_path, item)
    # Check if the item is a directory
    if os.path.isdir(clip_name):
        print("Directory:", clip_name)
    else:
        print("not a directory:", clip_name)
        continue
    
    actor_data_path = clip_name+'/actor_data'
    pc_data_path = clip_name+'/point_cloud'
    ego_data_path = clip_name+'/ego_data'

    actor_files = os.listdir(actor_data_path)
    pc_files = os.listdir(pc_data_path)
    ego_files = os.listdir(ego_data_path)

    actor_files.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
    pc_files.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
    ego_files.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))

    radius = 75 # lidar range in meters
    cube_size = .2 # size of each voxel in meters
    pixel_dims = int(2*radius/cube_size)+1 # number of voxels along each side of the grid

    bins = np.linspace(-radius, radius, pixel_dims) # bins for discretizing the point cloud

    # Deque to store the past 10 frames of lidar data along with the corresponding ego_pos
    # lidar_data_buffer = deque(maxlen=100)
    lidar_data_buffer = deque(maxlen=10)
    counter = 0

    # cv2.namedWindow('occupancy_im', cv2.WINDOW_NORMAL)
    cv2.namedWindow('dogma_vis', cv2.WINDOW_NORMAL)

    # Save the dogma to a numpy file in a directory with timestamp
    # timestamped_dir = create_timestamped_directory('./') # clip_name
    timestamped_dir = create_timestamped_directory(clip_name)
    ims = []
    for file_no, [pc_file, actor_file, ego_file] in enumerate(zip(pc_files, actor_files, ego_files)):
        print(f'processing captured data {file_no}/{len(pc_files)}')
        # file loading
        pc_file_path = os.path.join(pc_data_path, pc_file)
        actor_file_path = os.path.join(actor_data_path, actor_file)
        ego_file_path = os.path.join(ego_data_path, ego_file)

        pc_data = np.load(pc_file_path) # x, y, z, ObjIdx
        actor_data = np.load(actor_file_path) # frame_index, ObjIdx, transform, velocity, angular_velocity, acceleration, bounding_box
        ego_data = np.load(ego_file_path, allow_pickle=True) # frame_index, ObjIdx, transform, velocity, angular_velocity, acceleration, bounding_box

        if counter < 10: 
            counter += 1
            continue
        else: counter += 1

        # create a 2D grid of zeros for ego_frame
        local_grid = np.zeros((pixel_dims, pixel_dims))
        
        x_vels = np.zeros_like(local_grid)
        y_vels = np.zeros_like(local_grid)
        
        # retrieve the ego data
        ego_pos, ego_v, ego_bb = process_dynamic_actor(ego_data)
        # ego_pos_disc = discretize_coordinates(ego_pos[0], ego_pos[1], bins)
        ego_bb_disc = discretize_coordinates(ego_bb[:,0], ego_bb[:,1], bins)
        
        ego_mask = np.zeros_like(local_grid)
        cv2.fillPoly(ego_mask, [ego_bb_disc], color=(1))
        x_vels += ego_mask * ego_v[0]
        y_vels += ego_mask * ego_v[1]
        
        # for each actor in the scene
        for actor in actor_data:
            if actor[1] == ego_data[1]: continue
            # retrieve the actor data
            pos, v, bb = process_dynamic_actor(actor)
            offset = np.array(pos) - np.array(ego_pos)
            # discretize the actor bb into a 2D grid
            actor_bb_disc = discretize_coordinates(offset[0]+bb[:,0], offset[1]+bb[:,1], bins)
            # if the closest corner of the actor bb is not inside the grid, skip it
            distances = np.argmin(np.linalg.norm(actor_bb_disc[::-1] - np.array(ego_pos), axis=1))
            if np.any(actor_bb_disc[distances] <= 0) or np.any(actor_bb_disc[distances] >= pixel_dims): continue
            # create a mask for the actor bb
            actor_mask = np.zeros_like(local_grid)
            cv2.fillPoly(actor_mask, [actor_bb_disc], color=(1))
            local_grid += actor_mask
            x_vels += actor_mask * v[0]
            y_vels += actor_mask * v[1]
        
        # append the pc_coords and ego_pos to the lidar_data_buffer
        lidar_data_buffer.append((np.array([pc_data['x'], pc_data['y']]).T, np.array(ego_pos)))

        # create a set to hold the unique coordinates
        unique_coords = set()

        # get unique points from the deque
        for old_coords, old_pos in lidar_data_buffer:
            # transform the old coordinates to the current ego_pos frame
            transformed_coords = old_coords + (old_pos - ego_pos)
            # discretize the transformed coordinates
            transformed_coords_disc = np.unique(discretize_coordinates(transformed_coords[:, 0], transformed_coords[:, 1], bins), axis=0)
            for coord in transformed_coords_disc:
                unique_coords.add(tuple(coord))
        
        unique_coords_array = np.array(list(unique_coords))
        a, b = unique_coords_array[:, 0], unique_coords_array[:, 1]

        # Filter coordinates for display
        grid_size = local_grid.shape[0]
        valid_indices = np.logical_and(a > 0, b > 0)  # Remove negative indices
        a, b = a[valid_indices], b[valid_indices]
        valid_indices = np.logical_and(a < grid_size, b < grid_size)  # Remove indices larger than grid size
        a, b = a[valid_indices], b[valid_indices]
        local_grid[b, a] = 1

        # perform ray casting on the grid
        occupancy_grid = 1 - ray_trace_fast(local_grid)
        
        # # draw the ego bb on the grid
        occupancy_grid *= (1 - ego_mask)

        # stack the occupancy grid, x_vels, and y_vels
        dogma = np.stack((occupancy_grid, x_vels, y_vels), axis=-1)
        
        # save the dogma to a numpy file in a directory with timestamp
        np.save(os.path.join(timestamped_dir, f'dogma_{file_no}.npy'), dogma)
        
        # create a visualization of the dogma
        vel_im = create_rgb_image(dogma[:,:,1:], vmin=-10, vmax=10)
        occupancy_im = cv2.cvtColor((dogma[...,0]*255).astype(np.uint8), cv2.COLOR_GRAY2RGB)
        overlay = cv2.addWeighted(occupancy_im, 1, vel_im, 1, 0)
        
        ims.append(overlay)
        
        # use cv2 to display the grid
        cv2.imshow('dogma_vis', overlay)
        cv2.waitKey(1)
       
    cv2.destroyAllWindows()
    imageio.mimsave(clip_name+'/dogma_seq.mp4', ims, fps=60) # fps specifies frames per second


Directory: ./tmp/recording_2024-12-06_20-05-47
processing captured data 0/100
processing captured data 1/100
processing captured data 2/100
processing captured data 3/100
processing captured data 4/100
processing captured data 5/100
processing captured data 6/100
processing captured data 7/100
processing captured data 8/100
processing captured data 9/100
processing captured data 10/100
processing captured data 11/100
processing captured data 12/100
processing captured data 13/100
processing captured data 14/100
processing captured data 15/100
processing captured data 16/100
processing captured data 17/100
processing captured data 18/100
processing captured data 19/100
processing captured data 20/100
processing captured data 21/100
processing captured data 22/100
processing captured data 23/100
processing captured data 24/100
processing captured data 25/100
processing captured data 26/100
processing captured data 27/100
processing captured data 28/100
processing captured data 29/100
pro

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (751, 751) to (752, 752) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
